In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import imutils

In [2]:
img_yes = '../data/yes/'
img_no = '../data/no/'

yes_files = os.listdir(img_yes)
no_files = os.listdir(img_no)

yes_images, no_images = [], []
print('[!] Reading files with positive diagnostic...')
for file in yes_files:
    yes_images.append(plt.imread(f'{img_yes}{file}'))
print('[!] Positive files read!')

print('[!] Reading files with negative diagnostic...')
for file in no_files:
    no_images.append(plt.imread(f'{img_no}{file}'))
print('[!] Negative files read!')

all_images = yes_images + no_images

[!] Reading files with positive diagnostic...
[!] Positive files read!
[!] Reading files with negative diagnostic...
[!] Negative files read!


In [3]:
for el in all_images:
    print(el.dtype)
    if el.dtype == np.float32:
        print(el)

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
float32
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 

In [4]:
THRESH = 30
MAX_VALUE = 255

def convert_to_int_matrix(img_list):
    MIN, MAX = 0, 255
    for i, img in enumerate(img_list):
        if img.dtype == np.float32:
            tmp = np.copy(img) * MAX
            img_list[i] = tmp.astype(np.uint8)

def imgs_to_grayscale(img_list):
    for i, img in enumerate(img_list):
        if len(img.shape) == 3:
            img_list[i] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img_list):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.GaussianBlur(img, (5, 5), 0) 

def threshold_imgs(img_list):
    for i, img in enumerate(img_list):
        _, img_list[i] = cv2.threshold(img, THRESH, MAX_VALUE, cv2.THRESH_BINARY)

def erode_imgs(img_list):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.erode(img, None, iterations=1)

def dilate_imgs(img_list):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.dilate(img, None, iterations=1)

def flatten_contour(contour_list):
    tmp = contour_list[0]
    shape = tmp.shape
    aux = np.zeros(shape=(shape[0], shape[2]), dtype=int)
    for i in range(shape[0]):
        aux[i] = tmp[i].flatten()
    return aux

def find_extreme_points(contour_list):
    col_0 = contour_list[:, 0]
    col_1 = contour_list[:, 1]
    x_left, x_right = np.argmin(col_0), np.argmax(col_0)
    y_top, y_bot = np.argmax(col_1), np.argmin(col_1)
    ret = np.zeros(shape=(4, 2), dtype=int)
    ret[0, :], ret[1, :] = contour_list[x_left], contour_list[x_right]
    ret[2, :], ret[3, :] = contour_list[y_top], contour_list[y_bot]
    return ret  

def get_contour(img_list_morph, img_list):
    extreme_points, i = [], 0
    for img_morph, img in zip(img_list_morph, img_list):
        print(i)
        cont, _ = cv2.findContours(image=img_morph, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
        cont_flat = flatten_contour(cont)
        extreme_points.append(find_extreme_points(cont_flat))
        # print(extreme_points[-1])
        # img_copy = img.copy()
        # for i in range(4):
        #     img_copy = cv2.circle(img_copy, extreme_points[-1][i, :], radius=5, color=(0, 255, 0), thickness=-1)
        # cv2.imshow("Image", img_copy)
        # cv2.waitKey()
        # cv2.drawContours(image=img_copy, contours=extreme_points[-1], contourIdx=-1, color=(0, 255, 0), thickness=4, lineType=cv2.LINE_AA)
        # cv2.imshow("Image", img_copy)
        # cv2.waitKey()
        i+= 1
        #if i % 50 == 0: print(f'[!] {i} images processed...')
    return np.array(extreme_points)  

def draw_img(img_list, index):
    if index < len(img_list):
        cv2.imshow("Brain image", img_list[index])
        cv2.waitKey()

In [5]:
INDEX = 0
convert_to_int_matrix(all_images)
all_images_morph = all_images.copy()
draw_img(all_images, 0)
imgs_to_grayscale(all_images_morph)
draw_img(all_images_morph, INDEX)  
threshold_imgs(all_images_morph)
draw_img(all_images_morph, INDEX)
erode_imgs(all_images_morph)
draw_img(all_images_morph, INDEX)
dilate_imgs(all_images_morph)
draw_img(all_images_morph, INDEX)
gaussian_blur(all_images_morph)
draw_img(all_images_morph, INDEX)  
extreme_points = get_contour(all_images_morph, all_images)
extreme_points
#draw_img(all_images_morph, 0)  

uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint

KeyboardInterrupt: 